### INSTRUCTIONS AND RECOMMENDATIONS FOR CREATING YOUR OWN DATABASE FOR PHYLOGENETIC STUDIES

A database for phylogenetic study must contain:
- a BLAST database with the chosen proteomes;
- a file linking a gene to its protein products (g2r.tsv);
- a file linking a taxon identifier to scientific name of an organism (names.dmp).

We are going to get the most complete proteomes from the RefSeq database, so if you know you will be using your own set of proteomes, please, modify the algorithm accordingly. Moreover, we will need BLAST+ as well as Entrez Direct utilities.

Note that the algorithm is not intended for studying prokaryotic genes since these proteomes often use non-redundant protein accession numbers (with the prefix 'WP_').

#### 1. Getting the necessary files.

First, download the current proteomes list from RefSeq with the corresponding BUSCO completeness scores (you will need Entrez Direct installed).

In [22]:
%%bash

mkdir refseq_proteomes
cd refseq_proteomes
esearch -db assembly -query 'has_egap_annotation[prop] AND "latest refseq"[filter]' \
  | esummary \
  | xtract \
    -pattern DocumentSummary \
    -element \
      AssemblyAccession,\
      Organism,\
      Taxid,\
      Busco/BuscoLineage,\
      Busco/TotalCount,\
      Busco/Complete,\
      Busco/SingleCopy,\
      Busco/Duplicated,\
      Busco/Fragmented,\
      Busco/Missing \
> busco_scores.tsv

mkdir: cannot create directory ‘refseq_proteomes’: File exists


Then, get the taxdump.tar.gz.

In [4]:
%%bash

cd refseq_proteomes
mkdir taxonomy
cd taxonomy
wget -q ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz -O taxdump.tar.gz 
sleep 1
tar -xzf taxdump.tar.gz
rm taxdump.tar.gz
rm citations.dmp
rm delnodes.dmp
rm division.dmp
rm gencode.dmp
rm images.dmp
rm merged.dmp
rm readme.txt
rm gc.prt

citations.dmp
delnodes.dmp
division.dmp
gencode.dmp
images.dmp
merged.dmp
names.dmp
nodes.dmp
gc.prt
readme.txt


#### 2. Find the most complete proteomes.

We don't need to take all proteomes into the analysis: it is sufficient to take in several most complete proteomes from each class or another taxonomic rank.

We will take 3 most complete proteomes from classes.

Lets load nodes.dmp and names.dmp for managing taxonomic ranks:

In [5]:
import pandas as pd

sep = '\t\|\t'

nodes = pd.read_table(
    'refseq_proteomes/taxonomy/nodes.dmp',
    sep = sep,
    header = None,
    engine ='python'
)

nodes.columns = [
	'Taxid',
 	'Parent',
 	'Rank',
 	'EMBL',
 	'Division',
 	'Inherited_div',
 	'Gencode',
 	'Inherited_gencode',
 	'Mito',
 	'Inherited_mito',
 	'GenBank_hidden',
 	'Hidden_subtree',
 	'Comments'
]

names = pd.read_table(
    'refseq_proteomes/taxonomy/names.dmp',
    sep = sep,
    header = None,
    engine ='python'
)

names.columns = [
	'Taxid',
	'Name',
	'Unique',
	'Class'
]


Saving all identifiers of classes:

In [6]:
classes_ids = nodes[nodes['Rank'] == 'class']['Taxid'].to_list()

We also have to use a sorted (by a complete BUSCO score -- column 5 -- in our case) file with BUSCO scores:

In [23]:
busco = pd.read_table(
    'refseq_proteomes/busco_scores.tsv',
    header = None,
    sep = '\t'
)

busco.columns = [
    'GCF',
    'Name',
    'Taxid',
    'Lineage',
    'Count',
    'Score',
    'Single',
    'Dupl',
    'Fragm',
    'Miss'
]
busco = busco.dropna()
busco = busco.sort_values(by = ['Score'], ascending = False)

Lets use the taxid column as a list:

In [24]:
def find_class(current_taxid):
    if not (current_taxid in classes_ids):
        current_taxid = nodes[nodes['Taxid'] == current_taxid]['Parent'].to_list()
        if len(set(current_taxid)) > 1:
            raise Exception('Taxid', current_taxid, 'has multiple parents')
        elif len(set(current_taxid)) == 0:
            raise Exception('Warning! Root-tracing failed')
        elif current_taxid[0] == 1:
            return 0
        else:
            return find_class(current_taxid[0])
    else:
        return current_taxid

busco['Class'] = busco['Taxid'].map(find_class).astype(int)
print(len(busco['Class'].unique()))

36


There were 36 classes in total, and we are going to obtain up to 3*36=111 proteomes. Note that 0s indicate organisms that do not have class according to the nodes.dmp file. Lets see what these are.

In [25]:
busco[busco['Class'] == 0]

,GCF,Name,Taxid,Lineage,Count,Score,Single,Dupl,Fragm,Miss,Class
285,GCF_020497125.1,Mauremys mutica (yellowpond turtle),74926,sauropsida_odb10,7480,0.993182,0.971123,0.022059,0.001070,0.005749,0
441,GCF_016161935.1,Mauremys reevesii (Reeves's turtle),260615,sauropsida_odb10,7480,0.991578,0.975668,0.015909,0.001604,0.006818,0
74,GCF_027887155.1,Malaclemys terrapin pileata (diamondback terra...,2991368,sauropsida_odb10,7480,0.991444,0.979679,0.011765,0.001738,0.006818,0
323,GCF_015237465.2,Chelonia mydas (Green sea turtle),8469,sauropsida_odb10,7480,0.990107,0.978877,0.011230,0.003075,0.006818,0
635,GCF_007399415.2,Gopherus evgoodei (Goodes thornscrub tortoise),1825980,sauropsida_odb10,7480,0.989439,0.975134,0.014305,0.001604,0.008957,0
831,GCF_000281125.3,Alligator mississippiensis (American alligator),8496,sauropsida_odb10,7480,0.986364,0.981417,0.004947,0.006283,0.007353,0
132,GCF_025201925.1,Gopherus flavomarginatus (Mexican gopher torto...,286002,sauropsida_odb10,7480,0.984626,0.972193,0.012433,0.003342,0.012032,0
301,GCF_009764565.3,Dermochelys coriacea (leatherback sea turtle),27794,sauropsida_odb10,7480,0.982086,0.971791,0.010294,0.005214,0.012700,0
185,GCF_023653815.1,Caretta caretta (Loggerhead turtle),8467,sauropsida_odb10,7480,0.977941,0.967914,0.010027,0.002139,0.019920,0
532,GCF_013100865.1,Trachemys scripta elegans (red-eared slider tu...,31138,sauropsida_odb10,7480,0.975668,0.967112,0.008556,0.006283,0.018048,0


In the original paper, we have used other taxonomic ranks for these species. Now we are going to just take the top 3 proteomes with 0s, as well as Latimeria chalumnae and Protopterus annectens and human.

In [26]:
proteomes_list = [
    busco[busco['Taxid'] == 7897]['GCF'].to_list()[0],
    busco[busco['Taxid'] == 7888]['GCF'].to_list()[0],
    busco[busco['Taxid'] == 9606]['GCF'].to_list()[0]
]
# The top 3 with 0s will be added later

Now lets just take the top 3 proteomes in each class:

In [27]:
for name, class_df in busco.groupby('Class'):
    class_df = class_df.sort_values(by = ['Score'], ascending = False)
    class_proteomes = list()
    taxids = set()
    for index, row in class_df.iterrows():
        if not (row['Taxid'] in taxids):
            class_proteomes.append(row['GCF'])
        taxids.add(row['Taxid'])
    proteomes_list += class_proteomes[:3]

Lets write down obtained assembly identifiers.

In [28]:
with open('./refseq_proteomes/assembly_ids.txt', 'w') as out:
    out.write('\n'.join(list(set(proteomes_list))))

#### 3. Download proteomes, create and configure your BLAST database.

The NCBI Datasets tools might be used to download the proteomes but it is more convenient to use batch Entrez option.
- Go to https://www.ncbi.nlm.nih.gov/sites/batchentrez;
- Choose the Assembly Database;
- Select the "assembly_ids.txt" file;
- Click "Retrieve";
- Click "Retrieve records for ... UID(s)";
- Click "Download Assemblies", then choose the RefSeq database and "Protein FASTA (.faa)" file type;
- Create the "./refseq_proteomes/assembly_files" folder and place TAR files there;
- Click "Download Assemblies", then choose the RefSeq database and "Feature table (.txt)" file type;
- Place TAR files in the newly created "./refseq_proteomes/assembly_files" folder.

Lets manage the obtained files and concatenate all FASTA files and feature tables.

In [29]:
%%bash

cd ./refseq_proteomes/assembly_files
tar -xf genome_assemblies_features.tar
tar -xf genome_assemblies_prot_fasta.tar
mkdir ../busco_refseq

# Making a single FASTA file
> ../busco_refseq/busco_refseq.fasta
for fasta_file in ./*/*.faa.gz
do
  zcat $fasta_file >> ../busco_refseq/busco_refseq.fasta
done

# Header for feature table
for ft_file in ./*/*.txt.gz
do
  zcat $ft_file | head -n 1 | awk '{ \
      print $3,$11,$15,$16; \
    }' FS='\t' OFS='\t' \
    > ../busco_refseq/busco_feature_table.txt
  break
done

# Feature table
for ft_file in ./*/*.txt.gz
do
  zcat $ft_file | awk '{ \
      if ($1 == "CDS" && $2 == "with_protein") \
        print $3,$11,$15,$16; \
    }' FS='\t' OFS='\t' \
    >> ../busco_refseq/busco_feature_table.txt
done

mkdir: cannot create directory ‘../busco_refseq’: File exists


Checking if FASTA files and the feature table are fully compatible...

In [31]:
import os
import gzip
from glob import glob

ft = pd.read_csv('refseq_proteomes/busco_refseq/busco_feature_table.txt', sep = '\t')

assemblies = set()
for path in glob('refseq_proteomes/assembly_files/*/*.faa.gz'):
    filename = os.path.basename(path)
    assembly = '_'.join(filename.split('_')[0:2])
    assemblies.add(assembly)

if assemblies == set(ft['assembly']) == set(proteomes_list):
    print("Assemblies list: ok")
else:
    raise Exception("FASTA file and assembly list do not match")

for path in glob('refseq_proteomes/assembly_files/*/*.faa.gz'):
    filename = os.path.basename(path)
    with gzip.open(path, 'r') as inp:
        proteome = inp.readlines()
    proteome = [p.decode('utf-8').split()[0][1:] for p in proteome if p.startswith(b'>')]
    assembly = '_'.join(filename.split('_')[0:2])
    curr_df = ft[ft['assembly'] == assembly]
    if set(proteome) == set(curr_df['product_accession']):
        print(assembly + ': ok')
    else:
        raise Exception(assembly + ': protein accession numbers from the feature table and FASTA files do not match')

Assemblies list: ok
GCF_013753865.1: ok
GCF_028583425.1: ok
GCF_016835505.1: ok
GCF_001194135.2: ok
GCF_950005125.1: ok
GCF_914725855.1: ok
GCF_001039355.2: ok
GCF_018691265.1: ok
GCF_000001635.27: ok
GCF_007990345.1: ok
GCF_015342785.2: ok
GCF_000591075.1: ok
GCF_022113875.1: ok
GCF_000517525.1: ok
GCF_017312705.1: ok
GCF_002260705.1: ok
GCF_000633955.1: ok
GCF_020740725.1: ok
GCF_025612915.1: ok
GCF_011630105.1: ok
GCF_019649055.1: ok
GCF_000224145.3: ok
GCF_028641065.1: ok
GCF_009602425.1: ok
GCF_000003605.2: ok
GCF_006345805.1: ok
GCF_000003815.2: ok
GCF_000004255.2: ok
GCF_024679095.1: ok
GCF_019279795.1: ok
GCF_003286155.1: ok
GCF_021234035.1: ok
GCF_027172205.1: ok
GCF_000002075.1: ok
GCF_947568905.1: ok
GCF_010993605.1: ok
GCF_009764475.1: ok
GCF_947242115.1: ok
GCF_017976425.1: ok
GCF_017639785.1: ok
GCF_017639515.1: ok
GCF_020497125.1: ok
GCF_000485595.1: ok
GCF_022539665.1: ok
GCF_932526225.1: ok
GCF_018143015.1: ok
GCF_015706575.1: ok
GCF_900747795.2: ok
GCF_947563725.1: ok

Adding the taxonomy information to the feature table.

In [32]:
busco = busco[['GCF', 'Taxid']]

taxid_dict = dict()
for index, b in busco.iterrows():
    taxid_dict[b['GCF']] = b['Taxid']

ft['#tax_id'] = ft['assembly'].map(taxid_dict)

if len(ft['#tax_id'].unique()) != len(ft['assembly'].unique()):
    raise Exception('Check the tax_id and assembly correspondence')

g2r = ft

g2r = g2r.rename(columns = {
    'product_accession': 'protein_accession.version',
    'symbol': 'Symbol'
})

g2r.to_csv('refseq_proteomes/g2r.tsv', sep = '\t', index = False)

Finally, lets create a BLAST search database (provide path to your makeblastdb binary if necessary).

In [33]:
%%bash

cd ./refseq_proteomes/busco_refseq
makeblastdb \
  -dbtype prot \
  -in ./busco_refseq.fasta \
  -title busco_refseq \
  -parse_seqids \
  -out busco_refseq \
  -max_file_sz 2GB



Building a new DB, current time: 08/09/2023 15:16:20
New DB name:   /home/bioinfuser/applications/COG_diss/refseq_proteomes/busco_refseq/busco_refseq
New DB title:  busco_refseq
Sequence type: Protein


Deleted existing Protein BLAST database named /home/bioinfuser/applications/COG_diss/refseq_proteomes/busco_refseq/busco_refseq
Keep MBits: T
Maximum file size: 2000000000B
Adding sequences from FASTA; added 3298598 sequences in 155.5 seconds.


Edit the .ncbirc file accordingly (insert your paths if necessary).

In [34]:
%%bash

busco_refseq_path=$(readlink -f ./refseq_proteomes/busco_refseq)
echo "[BLAST]" > ~/.ncbirc
echo "BLASTDB=${busco_refseq_path}" >> ~/.ncbirc
echo "DATA_LOADERS=blastdb" >> ~/.ncbirc
echo "BLASTDB_PROT_DATA_LOADER=busco_refseq" >> ~/.ncbirc

Edit the configuration file (replace blastdbcmd_path and blastp_path if necessary).

In [1]:
%%bash

refseq_proteomes_path=$(readlink -f ./refseq_proteomes)
blastdbcmd_path=$(which blastdbcmd)
blastp_path=$(which blastp)
echo "# Local 'gene2refseq' file" > ./cogconf.txt
echo "path2G2R:${refseq_proteomes_path}/g2r.tsv" >> ./cogconf.txt
echo "# If you use refseq, download this file and unzip, then provide names.dmp file: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz" >> ./cogconf.txt
echo "path2T2N:${refseq_proteomes_path}/taxonomy/names.dmp" >> ./cogconf.txt
echo "# Name of database with representative taxids" >> ./cogconf.txt
echo "databaseName:busco_refseq" >> ./cogconf.txt
echo "# Path to Blastp utility" >> ./cogconf.txt
echo "path2blastp:${blastp_path}" >> ./cogconf.txt
echo "# Path to BlastDBCmd utility" >> ./cogconf.txt
echo "blastdbcmd:${blastdbcmd_path}" >> ./cogconf.txt

Your database is ready for work! Please, create a conda environment for this tool:
```
conda env create -n [NEW_ENV_NAME] -f dependencies.yml
```
...activate it...
```
conda activate [NEW_ENV_NAME]
```
 and test your installation:

In [2]:
%%bash

echo "NP_001104262" > ./refseq_proteomes/test_input.txt
echo "NP_001278" >> ./refseq_proteomes/test_input.txt
# you can observe the test_input.txt
python3 ./cog.py \
    ./refseq_proteomes/test_input.txt \
    ./refseq_proteomes/test_output

2023-08-10 14:46:12.493430: start
NP_001104262
NP_001278
2023-08-10 14:47:00.122005: "proteins" ready
2023-08-10 14:54:22.299444: Blast search completed (chunk 0)
2023-08-10 14:59:12.862464: Blast search completed (chunk 1)
2023-08-10 15:06:50.640761: Blast search completed (chunk 2)
2023-08-10 15:12:56.540567: Blast search completed (chunk 3)
2023-08-10 15:17:40.588645: Blast search completed (chunk 4)
2023-08-10 15:23:39.867273: Blast search completed (chunk 5)
2023-08-10 15:29:43.515175: Blast search completed (chunk 6)
2023-08-10 15:31:56.818472: Blast search completed (chunk 7)
Checking Blast dictionary...
2023-08-10 15:39:20.783133: Blast search completed (iteration 0, chunk 0)
2023-08-10 15:47:00.973461: Blast search completed (iteration 0, chunk 1)
2023-08-10 15:54:02.506368: Blast search completed (iteration 0, chunk 2)
2023-08-10 16:02:13.041004: Blast search completed (iteration 0, chunk 3)
2023-08-10 16:08:57.879575: Blast search completed (iteration 0, chunk 4)
2023-08-10 